# Cantilever Beam analyzed using Calculix and Code Aster

The following example demonstrates how you can use `ada` to quickly build, visualize, simulate and postprocess FEM analysis.

In [1]:
from ada.param_models.fem_models import beam_ex1

**Import an Assembly object from a function**

The function `beam_ex1()` builds and returns an `Assembly` containing a beam with a series of arbitrary cutouts. For more information on how the FEM model was created, you can look [here](https://github.com/Krande/adapy/blob/87f5356e519e6e707bfcde771e99e78225e27aff/src/ada/param_models/fem_models.py#L9-L59).

The `Assembly` method `to_ifc()` creates an ifc file that can be imported in most BIM software. My software of choice is the open source alternative using [blender](https://www.blender.org/) and the [blenderbim](https://blenderbim.org/) addon.

In [2]:
a = beam_ex1()
a.to_ifc('../output/MyCantilever.ifc')
a

Starting GMSH session
step file created at "C:\Users\ofskrand\ADA\temp\MyBeam_2g5QyY104Hx9OvyDM$DI$q.stp"
ifc file created at "..\output\MyCantilever.ifc"


Assembly("Test": Beams: 1, Plates: 0, Pipes: 0, Shapes: 0, Elements: 513, Nodes: 309)

**Run a FEM analysis using Calculix**

The Assembly method `to_fem` creates an analysis directory called "MyCantilever_calculix" inside the `scratch`
directory.

By passing in `execute=True` calculix and code_aster will automatically start the analysis once the input files are
created.

In [3]:
ccx_name = "MyCantilever_calculix"
a.to_fem(ccx_name, "calculix", overwrite=True, execute=True)

Created a Calculix input deck at "C:\AibelProgs\ADA\scratch\MyCantilever_calculix"
--------------------------------------------------------------------------------
starting Calculix simulation "MyCantilever_calculix"
Finished Calculix simulation "MyCantilever_calculix"
--------------------------------------------------------------------------------


ERROR:root:FEM format "calculix" is not yet supported for results processing


**Run a FEM analysis using Code Aster**

In [4]:
ca_name = "MyCantilever_code_aster"
a.to_fem(ca_name, "code_aster", overwrite=True, execute=True)

Created a Code_Aster input deck at "C:\AibelProgs\ADA\scratch\MyCantilever_code_aster"
--------------------------------------------------------------------------------
starting CodeAsterAnalysis simulation "MyCantilever_code_aster"
Finished CodeAsterAnalysis simulation "MyCantilever_code_aster"
--------------------------------------------------------------------------------


Results(code_aster, MyCantilever_code_aster.rmed)

**Reading the analysis results using Python**

* Calculix:
The analysis result file (frd) was automatically converted to VTU using the
[ccx2paraview](https://github.com/calculix/ccx2paraview) package.

* Code Aster:
The analysis result file (RMED) can be read directly using meshio.


By using meshio to read the `VTU` and `RMED` files (from Calculix and Code Aster respectively),
it is possible to access the result data such as the nodal displacements 'U' (as shown below).

In [5]:
from ada.config import Settings
import meshio

vtu = (Settings.scratch_dir / ccx_name / ccx_name).with_suffix(".vtu")
ccx_mesh = meshio.read(vtu)

# Displacements in [X, Y, Z] at point @ index=-1

print(ccx_mesh.point_data['U'][-1])

[-0.00100044  0.00246466 -0.0202001 ]


In [6]:
rmed = (Settings.scratch_dir / ca_name / ca_name).with_suffix(".rmed")
ca_mesh = meshio.read(rmed, 'med')

# Displacements in [X, Y, Z] at point @ index=-1
print(ca_mesh.point_data['DISP[10] - 1'][-1][:3])

[-0.0010598  -0.00010826 -0.02187207]
